This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

im1 = itk.imread("../Data/MRI-Cases/mra.mha", itk.F)

resampler = ttk.ResampleImage.New( Input=im1, MakeHighResIso=True )
resampler.Update()
im1iso = resampler.GetOutput()


In [3]:
view(im1iso)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [4]:
imMath = ttk.ImageMath[ImageType,ImageType].New( Input=im1iso )
imMath.Blur(1)
imBlur = imMath.GetOutput()

numSeeds = 10

In [5]:
vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.0)
vSeg.SetMinRoundness(0.1)
vSeg.SetRadiusInObjectSpace( 1 )
vSeg.SetSeedMask( imBlur )
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )

vSeg.ProcessSeeds()

In [6]:
tubeMaskImage = vSeg.GetTubeMaskImage()
view(tubeMaskImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [8]:
SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
SOWriter.SetFileName( "Demo-SegmentBrightVessels.tre" )
SOWriter.Update()